**Aflevering 2 - Opgave 2.2**

In [1]:
pip install git+https://github.com/alemartinello/dstapi.git

  Cloning https://github.com/alemartinello/dstapi.git to /private/var/folders/71/b9kzpv3s4w5fd9kz2fr224x40000gn/T/pip-req-build-sjm_nyi7
  Running command git clone --filter=blob:none --quiet https://github.com/alemartinello/dstapi.git /private/var/folders/71/b9kzpv3s4w5fd9kz2fr224x40000gn/T/pip-req-build-sjm_nyi7
  Resolved https://github.com/alemartinello/dstapi.git to commit d9eeb5a82cbc70b7d63b2ff44d92632fd77123a4
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas_datareader


Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import dstapi as dst
import matplotlib.pyplot as plt
#from fredapi import Fred
from pandas_datareader import data as pdr
from IPython.display import display

**Opgave 2.2.1**

In [4]:
plt.rcParams.update({'axes.grid':True,'grid.color':'black','grid.alpha':0.25,'grid.linestyle':'--','font.size':13})


In [11]:
# Henter HICP fra FRED
hicp = pdr.DataReader('CP0000DKM086NEST', 'fred')
hicp = hicp.reset_index().rename(columns={'DATE': 'date', 'CP0000DKM086NEST': 'HICP'})

print(f"Antal rækker hentet fra FRED: {len(hicp)}")
display(hicp.head())


Antal rækker hentet fra FRED: 59


,date,HICP
0,2020-11-01,103.0
1,2020-12-01,102.8
2,2021-01-01,103.0
3,2021-02-01,103.5
4,2021-03-01,103.6


In [16]:
fred_ids = {
    'Denmark'      : 'CP0000DKM086NEST',  # HICP
    'Austria'      : 'CP0000ATM086NEST',  # HICP
    'Euro Area'    : 'CP0000EZM086NEST',  # HICP
    'United States': 'CP0000USM086NEST'   # HICP
}

dfs = []
for country, sid in fred_ids.items():
    s = fred.get_series(sid, observation_start='2019-01-01')  # henter som pandas.Series
    df = s.rename('value').to_frame().reset_index().rename(columns={'index':'date'})
    df['country'] = country
    dfs.append(df)

hicp_long = pd.concat(dfs, ignore_index=True).sort_values(['country','date']).reset_index(drop=True)
print(hicp_long.head())

NameError: name 'fred' is not defined